In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from matplotlib import patches

import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
default_b = 2.1
default_a = 1

In [ ]:
def read_network():
    w1 = np.loadtxt('w1.txt')
    w2 = np.loadtxt('w2.txt')
    w3 = np.loadtxt('w3.txt')
    b1 = np.loadtxt('b1.txt')
    b2 = np.loadtxt('b2.txt')
    b3 = np.loadtxt('b3.txt')
    return w1, w2, w3, b1, b2, b3

In [ ]:
w1, w2, w3, b1, b2, b3 = read_network()

In [ ]:
state = np.loadtxt('state.txt')

In [ ]:
def np2ten(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float64)
    return arg

In [ ]:
x = tf.placeholder(tf.float64, [None, 3], name='x')
y_ = tf.placeholder(tf.float64, [None, 2], name='y_')

w1 = np2ten(w1)
w2 = np2ten(w2)
w3 = np2ten(w3)
b1 = np2ten(b1)
b2 = np2ten(b2)
b3 = np2ten(b3)

l1 = tf.nn.relu(tf.matmul(x, w1) + b1)
l2 = tf.nn.relu(tf.matmul(l1, w2) + b2)
y = tf.matmul(l2, w3) + b3

rows = [tf.gradients(y[:, i], x)[0] for i in range(int(y.shape[1]))]
J2 = tf.stack(rows, axis=1)

sess = tf.InteractiveSession()

In [ ]:
def brusselator_approx(x_array, b=default_b):
    x_array = np.asarray(x_array).reshape((-1, 2))
    b = np.ones([len(x_array), 1]) * b
    x_array = np.hstack([x_array, b])
    return sess.run(y, feed_dict={x: x_array})

In [ ]:
def J0(b):

    N = 1501
    x_grid = np.linspace(-5, 10, N)
    y_grid = np.linspace(-5, 10, N)
    x_grid, y_grid = np.meshgrid(x_grid, y_grid)

    b_array = np.ones_like(x_grid.ravel()) * b

    grid_ravel = np.vstack([x_grid.ravel(), y_grid.ravel(), b_array]).T
    grid_image = sess.run(y, feed_dict={x: grid_ravel})
    jac = sess.run(J2, feed_dict={x: grid_ravel, y:grid_image})

    jac = np.linalg.det(jac[:, :, :-1])
    jac = jac.reshape((N, N))
    
    ind = np.where((jac[:, :-1] * jac[:, 1:]) < 0)
    J0_x = x_grid[[ind[0], ind[1]]]
    J0_y = y_grid[[ind[0], ind[1]]]
    return np.array([J0_x, J0_y])

In [ ]:
J0_points = J0(default_b)

In [ ]:
points = np.array([[1.96433, 1.24951],
                   [0.666201, 1.99221]])

In [ ]:
r = [0.655344127938151, 1.18737691940507]
R = [2.77382102844230, 4.23615564741905]

invs = np.array([[4.73825188, -1.10571284], 
                 [8.83688484, -4.29014662],
                 [-3.56999818, 5.46926724], 
                 [-25.79530631, 21.67496866]])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6), ncols=2)
for i in range(2):
    ax[i].scatter(J0_points[0], J0_points[1], s=5, label=r'$J_0$', color='red')
    
    ax[i].scatter(*state.T, color='orange', label='Attractor')
    
    ax[i].scatter(*brusselator_approx(points[i]).ravel(), marker='^',s=100, color='black', label='Image')
    
    ax[i].scatter(*points[i].ravel(), marker='+',s=100, color='blue', label='Inverses', zorder=99)
    
    ax[i].scatter(*invs[2 * i].ravel(), marker='+',s=100, color='blue', zorder=99)
    ax[i].scatter(*invs[2 * i + 1].ravel(), marker='+',s=100, color='blue', zorder=99)
    
    square1 = patches.Rectangle((points[i, 0] - r[i], points[i, 1] - r[i]), 2 * r[i], 2 * r[i], fill=False, edgecolor='k')
    ax[i].add_patch(square1)
    
    square2 = patches.Rectangle((points[i, 0] - R[i], points[i, 1] - R[i]), 2 * R[i], 2 * R[i], fill=False, edgecolor='k')
    ax[i].add_patch(square2)
   
    ax[i].set_xlim(-5, 10)
    ax[i].set_ylim(-5, 10)
    
    ax[i].tick_params(axis='x', labelsize=16)
    ax[0].tick_params(axis='y', labelsize=16)
    
    ax[1].set_yticklabels(['' for y in ax[1].get_yticks()])
    
    if not i:
        ax[i].set_ylabel('$y$', fontsize=24)
    
    ax[i].set_xlabel('$x$', fontsize=24)
    ax[0].legend(fontsize=16)

plt.tight_layout()

# plt.savefig('2d_point.pdf', bbox_inches='tight')

# plt.show()